In [1]:
from selenium import webdriver 
from selenium.webdriver.support.ui import Select
import time
import sys
from selenium.webdriver.common.keys import Keys
from selenium.webdriver import ActionChains
import pandas as pd

In [2]:
exeDriver = "chromedriver.exe"
driver = webdriver.Chrome(exeDriver)
print("Iniciando Scraper")

Iniciando Scraper


In [3]:
url = "http://procomun.educalab.es/es"
driver.get(url)

### Busqueda y filtrado por recursos de aprendizaje

In [4]:
content_type = driver.find_element_by_id('edit_type_chosen')
content_type_link = content_type.find_element_by_class_name("chosen-single")
content_type_link.click()

options = content_type.find_elements_by_class_name("active-result")
learning_object = options[1]
learning_object.click()
button = driver.find_element_by_id("new-search")
button.click()
time.sleep(5)
#driver.implicitly_wait(2)

### Obteniendo metadatos articulos

In [5]:
def get_metadata_page():
    URL = driver.current_url
    
    TITULO = driver.find_element_by_xpath('/html/body/section/div/div/div[1]/div/span[5]')
    TITULO = TITULO.text
    
    AUTORES = driver.find_element_by_class_name('author')
    AUTORES = AUTORES.text
    
    FECHA = driver.find_element_by_class_name('detail-ode-publish')
    FECHA = FECHA.text

    PALABRAS_CLAVE = ''
    try:
        general = driver.find_element_by_xpath('/html/body/section/div/div/div[2]/div/section/div/section/div/div/div/div[2]/div[2]/div[1]/div[1]/div')
        general.click()
        palabras = general.find_element_by_xpath('/html/body/section/div/div/div[2]/div/section/div/section/div/div/div/div[2]/div[2]/div[1]/div[2]/div/div/div')
        palabras = palabras.text
        palabras_list = palabras.split('\n')
        PALABRAS_CLAVE = palabras_list
    except:
        pass
    
    # Seccion principal
    RESUMEN = ''
    ORIENTACIONES_DIDACTICAS = ''
    LICENCIA = ''
    TIPO_DE_LICENCIA = ''
    USUARIO_FINAL = ''
    CONTEXTO_EDUCATIVO = ''
    seccion_metadatos = driver.find_element_by_id('block-system-main')
    metadatos = seccion_metadatos.find_element_by_xpath('//*[@id="block-system-main"]/div/div/div[2]/div[1]')
    metadatos_txt = metadatos.text
    metadatos_list = metadatos_txt.split('\n')
    for i in range(len(metadatos_list)):
        if (metadatos_list[i] == 'RESUMEN'):
            for j in range(i+1, len(metadatos_list)):
                if ((metadatos_list[j] == 'ORIENTACIONES DIDÁCTICAS') or (metadatos_list[j] == 'LICENCIA')):
                    break
                else:
                    RESUMEN += metadatos_list[j] + ' '
        if (metadatos_list[i] == 'ORIENTACIONES DIDÁCTICAS'):
            for j in range(i+1, len(metadatos_list)):
                if (metadatos_list[j] == 'LICENCIA'):
                    break
                else:
                    ORIENTACIONES_DIDACTICAS += metadatos_list[j] + ' '
        if (metadatos_list[i] == 'LICENCIA'):
            LICENCIA = metadatos_list[i+1]
        if (metadatos_list[i] == 'TIPO DE LICENCIA'):
            TIPO_DE_LICENCIA = metadatos_list[i+1]
        if (metadatos_list[i] == 'USUARIO FINAL'):
            USUARIO_FINAL = metadatos_list[i+1]
        if (metadatos_list[i] == 'CONTEXTO EDUCATIVO'):
            CONTEXTO_EDUCATIVO = metadatos_list[i+1]
        
    # Seccion USO EDUCATIVO 
    USO_EDUCATIVO_EDAD = ''
    USO_EDUCATIVO_IDIOMA_DESTINATARIO = ''
    USO_EDUCATIVO_PROCESO_COGNITIVO = ''
    USO_EDUCATIVO_TIPO_DE_INTERACTIVIDAD = ''
    usoEducativo = driver.find_element_by_xpath('/html/body/section/div/div/div[2]/div/section/div/section/div/div/div/div[2]/div[2]/div[3]/div[1]/div')
    usoEducativo.click()
    elementos = usoEducativo.find_element_by_xpath('/html/body/section/div/div/div[2]/div/section/div/section/div/div/div/div[2]/div[2]/div[3]')
    elementos = elementos.text
    elementos_list = elementos.split('\n')
    length = len(elementos_list)
    if (length > 1):
        for i in range(1, length):
            if (elementos_list[i] == 'EDAD'):
                USO_EDUCATIVO_EDAD = elementos_list[i+1]
            if (elementos_list[i] == 'IDIOMA DESTINATARIO'):
                USO_EDUCATIVO_IDIOMA_DESTINATARIO = elementos_list[i+1]
            if (elementos_list[i] == 'PROCESO COGNITIVO'):
                USO_EDUCATIVO_PROCESO_COGNITIVO = elementos_list[i+1]
            if (elementos_list[i] == 'TIPO DE INTERACTIVIDAD'):
                USO_EDUCATIVO_TIPO_DE_INTERACTIVIDAD = elementos_list[i+1]
                
    # Seccion tecnica
    TECNICA_PLATAFORMA = ''
    try:
        tecnica = driver.find_element_by_xpath('/html/body/section/div/div/div[2]/div/section/div/section/div/div/div/div[2]/div[2]/div[2]/div[1]/div')
        tecnica.click()
        elementosTecnica = tecnica.find_element_by_xpath('/html/body/section/div/div/div[2]/div/section/div/section/div/div/div/div[2]/div[2]/div[2]/div[2]/div/div/div')
        TECNICA_PLATAFORMA = elementosTecnica.text
    except:
        pass
    
    # Seccion clasificacion
    CLASIFICACION_AREA_DE_CONOCIMIENTO = ''
    CLASIFICACION_CONTEXTO_EDUCATIVO = ''
    CLASIFICACION_TIPO_DE_RECURSO = ''
    clasificacion = driver.find_element_by_xpath('/html/body/section/div/div/div[2]/div/section/div/section/div/div/div/div[2]/div[2]/div[4]/div[1]/div')
    clasificacion.click()
    elemClasificacion = clasificacion.find_element_by_xpath('/html/body/section/div/div/div[2]/div/section/div/section/div/div/div/div[2]/div[2]/div[4]')
    elemClasificacion = elemClasificacion.text
    elemClasificacion_list = elemClasificacion.split('\n')
    length = len(elemClasificacion_list)
    if (length > 1):
        for i in range(1, length):
            if (elemClasificacion_list[i] == 'TIPO DE RECURSO'):
                CLASIFICACION_TIPO_DE_RECURSO = elemClasificacion_list[i+1]
            if (elemClasificacion_list[i] == 'ÁREA DE CONOCIMIENTO'):
                CLASIFICACION_AREA_DE_CONOCIMIENTO = elemClasificacion_list[i+1]
            if (elemClasificacion_list[i] == 'CONTEXTO EDUCATIVO'):
                CLASIFICACION_CONTEXTO_EDUCATIVO = elemClasificacion_list[i+1]

    # Agregando a df
    diccionario = {'TITULO': TITULO, 'PALABRAS_CLAVE': PALABRAS_CLAVE, 'RESUMEN': RESUMEN, 
                   'ORIENTACIONES_DIDACTICAS': ORIENTACIONES_DIDACTICAS, 'LICENCIA': LICENCIA, 
                   'TIPO_DE_LICENCIA': TIPO_DE_LICENCIA, 'AUTORES': AUTORES, 'USUARIO_FINAL': USUARIO_FINAL, 
                   'CONTEXTO_EDUCATIVO': CONTEXTO_EDUCATIVO, 'FECHA': FECHA, 'USO_EDUCATIVO_EDAD': USO_EDUCATIVO_EDAD, 
                   'USO_EDUCATIVO_IDIOMA_DESTINATARIO': USO_EDUCATIVO_IDIOMA_DESTINATARIO, 'USO_EDUCATIVO_PROCESO_COGNITIVO': USO_EDUCATIVO_PROCESO_COGNITIVO,
                   'USO_EDUCATIVO_TIPO_DE_INTERACTIVIDAD': USO_EDUCATIVO_TIPO_DE_INTERACTIVIDAD, 'URL': URL, 'TECNICA_PLATAFORMA': TECNICA_PLATAFORMA,
                   'CLASIFICACION_AREA_DE_CONOCIMIENTO': CLASIFICACION_AREA_DE_CONOCIMIENTO, 'CLASIFICACION_CONTEXTO_EDUCATIVO': CLASIFICACION_CONTEXTO_EDUCATIVO, 
                   'CLASIFICACION_TIPO_DE_RECURSO': CLASIFICACION_TIPO_DE_RECURSO}
    return diccionario

In [23]:
columns = ['TITULO', 'PALABRAS_CLAVE', 'AUTORES', 'FECHA', 'RESUMEN', 'ORIENTACIONES_DIDACTICAS', 
           'LICENCIA', 'TIPO_DE_LICENCIA', 'USUARIO_FINAL', 'CONTEXTO_EDUCATIVO', 'USO_EDUCATIVO_EDAD',
           'USO_EDUCATIVO_IDIOMA_DESTINATARIO', 'USO_EDUCATIVO_TIPO_DE_INTERACTIVIDAD', 
           'USO_EDUCATIVO_PROCESO_COGNITIVO','TECNICA_PLATAFORMA', 'CLASIFICACION_AREA_DE_CONOCIMIENTO', 
           'CLASIFICACION_CONTEXTO_EDUCATIVO', 'CLASIFICACION_TIPO_DE_RECURSO', 'URL']
df = pd.DataFrame(columns=columns)

for i in range(0, 1000): # Itera sobre paginas
    for j in range(1, 11): # Itera sobre articulos
        try:
            articulo1 = driver.find_element_by_xpath('/html/body/section/div/div/div/section/div/section/div[2]/ul/li[' + str(j) + ']/div[1]/h2/a')
            articulo1.click()
            driver.switch_to.window(driver.window_handles[-1])

            diccionario = get_metadata_page()   
            df = df.append(diccionario, ignore_index=True)

            driver.close()
            driver.switch_to.window(driver.window_handles[0])
        except:
            print('Fallo la pagina ' + str(i) + ' el articulo ' + str(j))
            pass

    try:
        print('Van: ' + str(i) + ' paginas')
        next_page = driver.find_element_by_class_name("pager-next")
        next_page.click()
        time.sleep(5)
        #driver.implicitly_wait(2)
    except:
        print("Fallo al pasar de pagina")
        pass

Fallo la pagina 797 el articulo 1
Van: 797 paginas
Van: 798 paginas
Van: 799 paginas
Van: 800 paginas
Van: 801 paginas
Van: 802 paginas
Van: 803 paginas
Van: 804 paginas
Van: 805 paginas
Van: 806 paginas
Van: 807 paginas
Van: 808 paginas
Van: 809 paginas
Van: 810 paginas
Van: 811 paginas
Van: 812 paginas
Van: 813 paginas
Van: 814 paginas
Van: 815 paginas
Van: 816 paginas
Van: 817 paginas
Van: 818 paginas
Van: 819 paginas
Van: 820 paginas
Van: 821 paginas
Van: 822 paginas
Van: 823 paginas
Van: 824 paginas
Van: 825 paginas
Van: 826 paginas
Van: 827 paginas
Van: 828 paginas
Van: 829 paginas
Van: 830 paginas
Van: 831 paginas
Van: 832 paginas
Van: 833 paginas
Van: 834 paginas
Van: 835 paginas
Van: 836 paginas
Van: 837 paginas
Van: 838 paginas
Van: 839 paginas
Van: 840 paginas
Van: 841 paginas
Van: 842 paginas
Van: 843 paginas
Van: 844 paginas
Van: 845 paginas
Van: 846 paginas
Van: 847 paginas
Van: 848 paginas
Van: 849 paginas
Van: 850 paginas
Van: 851 paginas
Van: 852 paginas
Van: 853 pagin

In [24]:
df

TITULO  \
0                        How they live: Home sweet home   
1         How they live: Elementos comunes de la unidad   
2                   Culture Vultures: The plot thickens   
3                            2015, la boutique du futur   
4              Querido diario (La Feria de la Historia)   
5     Restauración II y Primo de Rivera (La Feria de...   
6                                       Días de trabajo   
7                                     Cocina Fría CESDE   
8      Culture Vultures: Elementos comunes de la unidad   
9                   Culture Vultures: The silver screen   
10            Culture Vultures: All the world's a stage   
11                                    Examen de Energía   
12                          Acción o reacción (química)   
13                      Interacción Oral: Je suis perdu   
14               Interacción Escrita: "Je suis épuisée"   
15              Ikusteko urritasuna - Kirola egiten dut   
16                             Expresión Oral: L'amitié   
17                                 Lurra eta unibertsoa   
18          Expresión Escrita: Les relations au travail   
19               Ikusteko urritasuna - Eguneroko gauzak   
20                        Ikusteko urritasuna - Eskolan   
21                        SD 6 - Protección del entorno   
22      Interacción Oral: Je ne supporte plus mon coloc   
23                   Interacción Escrita: Centre de Tri   
24                Expresión Oral: changement climatique   
25              Expresión Escrita: Empreinte écologique   
26    Guía Atención Educativa al Alumnado con Discap...   
27                                     The first people   
28                            Spain is not so different   
29                               They drive on the left   
...                                                 ...   
1999                               Book of instructions   
2000     Migration flows, rural depopulation and ageing   
2001                      Els sentits i els seus òrgans   
2002                                        Discalculia   
2003                                            Numbers   
2004                About me. Material para el alumnado   
2005                       Learning the names of people   
2006  Coordenadas terrestres. Material para el alumnado   
2007                                Concepto de materia   
2008                                Inteligencia límite   
2009                                           El guión   
2010  Sistema métrico decimal. Material para el alum...   
2011                        Zentzuak eta beren organoak   
2012                                   Higiene postural   
2013                                    Motricitat fina   
2014  Poliedros y cuerpos de revolución. Material pa...   
2015            Las máquinas. Material para el alumnado   
2016                                        Simple past   
2017                                      Make pancakes   
2018            Las horas, los minutos, los segundos...   
2019                                        El comercio   
2020                          El azar y la probabilidad   
2021  ESTUDIO DE LOS HONGOS DE INTERÉS COMERCIAL DEL...   
2022           Arazoa hauteman eta irtenbidea bilatzea.   
2023                                El cambio climático   
2024                             La grabadora de medios   
2025                                      Biodiversidad   
2026              Agrupaciones musicales instrumentales   
2027                  Identifying people by their names   
2028     La població espanyola. Migracions i estructura   

                                         PALABRAS_CLAVE               AUTORES  \
0                 [mda, causalidad, relativos, hogares]    Junta de Andalucía   
1         [mda, condicionales, superlativos, relativos]    Junta de Andalucía   
2               [mda, literatura, prefijos, conectores]    Junta de Andalucía   
3     [mda, competencia comunicativa, enseñanza en l...           

In [25]:
df.to_excel("dataEXCEL5.xlsx", "dataEXCEL")
try:
    df.to_csv("data5.csv", sep = ';', encoding = 'ISO-8859-1')
except:
    df.to_csv("dataNoEncode5.csv", sep = ';')